In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
import tensorflow.keras

In [2]:
train_dir='Documents/Watanabe_lab/Research_data/DeepLearning/train'
valid_dir='Documents/Watanabe_lab/Research_data/DeepLearning/validation'
test_dir='Documents/Watanabe_lab/Research_data/DeepLearning/test'

In [3]:
conv_base=VGG16(weights='imagenet',
               include_top=False,
               input_shape=(200,200,3))

In [4]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [5]:
datagen=ImageDataGenerator(rescale=1/255)
batch_size=20

def extract_features(directory,sample_count):
    features=np.zeros(shape=(sample_count,6,6,512))
    labels=np.zeros(shape=(sample_count))
    generator=datagen.flow_from_directory(directory,target_size=(200,200),
                                         batch_size=batch_size,
                                         class_mode='binary')
    
    i=0
    for input_batch,labels_batch in generator:
        features_batch=conv_base.predict(input_batch)
        features[i*batch_size:(i+1)*batch_size]=features_batch
        labels[i*batch_size:(i+1)*batch_size]=labels_batch
        i+=1
        if i*batch_size>=sample_count:
            break
            
    
    return features,labels

In [6]:
train_features,train_labels=extract_features(train_dir,285)
val_features,val_labels=extract_features(valid_dir,106)
test_features,test_labels=extract_features(test_dir,99)

Found 285 images belonging to 2 classes.
Found 106 images belonging to 2 classes.
Found 99 images belonging to 2 classes.


In [7]:
train_features=np.reshape(train_features,(285,6*6*512))
val_features=np.reshape(val_features,(106,6*6*512))
test_features=np.reshape(test_features,(99,6*6*512))

In [8]:
model=models.Sequential()
model.add(layers.Dense(256,activation='relu',input_dim=6*6*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['acc'])

callbacks=[
    tensorflow.keras.callbacks.EarlyStopping(
    monitor='val_acc',
    patience=10),
    
    tensorflow.keras.callbacks.TensorBoard(
    log_dir='my_log_dir',
    histogram_freq=1,
    embeddings_freq=1)]

history=model.fit(train_features,train_labels,
                 epochs=50,
                 batch_size=20,
                 validation_data=(val_features,val_labels),
                 callbacks=callbacks)

Epoch 1/50
15/15 [==============================] - 1s 58ms/step - loss: 0.6461 - acc: 0.6632 - val_loss: 0.3717 - val_acc: 0.8491
Epoch 2/50
15/15 [==============================] - 1s 47ms/step - loss: 0.5914 - acc: 0.7228 - val_loss: 0.3970 - val_acc: 0.8679
Epoch 3/50
15/15 [==============================] - 1s 49ms/step - loss: 0.4971 - acc: 0.7895 - val_loss: 0.4203 - val_acc: 0.8962
Epoch 4/50
15/15 [==============================] - 1s 47ms/step - loss: 0.4627 - acc: 0.8000 - val_loss: 0.3446 - val_acc: 0.8774
Epoch 5/50
15/15 [==============================] - 1s 50ms/step - loss: 0.4421 - acc: 0.8070 - val_loss: 0.3652 - val_acc: 0.8868
Epoch 6/50
15/15 [==============================] - 1s 52ms/step - loss: 0.4125 - acc: 0.8211 - val_loss: 0.3508 - val_acc: 0.8774
Epoch 7/50
15/15 [==============================] - 1s 50ms/step - loss: 0.3692 - acc: 0.8316 - val_loss: 0.3112 - val_acc: 0.8679
Epoch 8/50
15/15 [==============================] - 1s 47ms/step - loss: 0.3461 - a

In [ ]:
!tensorboard --logdir=my_log_dir

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.2.1 at http://localhost:6006/ (Press CTRL+C to quit)
